In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/My Drive/cifar10_project')
os.getcwd()

'/content/drive/My Drive/cifar10_project'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

W0817 11:18:40.654163 139739535169408 deprecation.py:323] From <ipython-input-3-46c62fe1e279>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0817 11:18:40.655506 139739535169408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0817 11:18:40.658789 139739535169408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0817 11:18:40.930082 139739535169408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0817 11:18:40.935473 139739535169408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0817 11:18:40.986994 139739535169408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
import tensorflow as tf

def conv2d(inputs,filters, kernel_size = 3, strides = 1, activation = None, batch_norm = None):
    x = tf.layers.conv2d(inputs, filters = filters, kernel_size=kernel_size, strides=strides, padding = 'SAME',activation=None)
    if batch_norm:   
        x = tf.layers.batch_normalization(x)
    x = activation(x)
    return x
    
def conv2d_transpose(inputs, filters, kernel_size = 2, strides = 2, activation = None, batch_norm = None):
    x = tf.layers.conv2d_transpose(inputs, filters = filters, kernel_size = kernel_size, strides = strides, padding='SAME', activation = None)
    if batch_norm:      
        x = tf.layers.batch_normalization(x)
    x = activation(x)
    return x
  
  
class GAN_cnn():
    def __init__(self, input_shape = [28,28,1], n_noise=128):
    
        self.input_shape = input_shape
        self.n_noise = n_noise
        
        self.h, self.w = self.input_shape[0]//4, self.input_shape[1]//4
        assert self.h*4 == self.input_shape[0] and self.w*4 ==self.input_shape[1], 'input_shape이 적절하지 않습니다.'
                
    def get_noise(self, batch_size):
        return np.random.uniform(-1.,1., size = [batch_size, self.n_noise])

    def generator(self, noise, labels):
        with tf.variable_scope('generator',reuse = tf.AUTO_REUSE):
            
            inputs = tf.concat([noise,labels], 1)

            x = tf.layers.dense(inputs, self.h*self.w*256, activation = tf.nn.relu)
            x = tf.reshape(x, shape = [-1, self.h, self.w, 256])
                     
            x = conv2d_transpose(x, filters = 128, kernel_size = 2, strides = 2, activation = tf.nn.relu)
            x = conv2d_transpose(x, filters = 64, kernel_size = 2, strides = 2, activation = tf.nn.relu)
            x = conv2d(x, filters = self.input_shape[-1], kernel_size = 3, strides = 1, activation = tf.nn.sigmoid)
            return x
        
    def discriminator(self, inputs, labels):
        with tf.variable_scope('discriminator',reuse = tf.AUTO_REUSE):

            x = tf.layers.flatten(inputs)
            x = tf.concat([x, labels], 1)
            x = tf.layers.dense(x, 256, activation = tf.nn.relu)
            x = tf.layers.dense(x, 1, activation = None)
            
            return x
 

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.reset_default_graph()

input_shape = [28,28,1]
n_class = 10
n_noise=128

model = GAN_cnn(input_shape = input_shape, n_noise = n_noise)


X = tf.placeholder(tf.float32, [None]+input_shape)
Y = tf.placeholder(tf.float32, [None,n_class])
Z = tf.placeholder(tf.float32, [None,n_noise])

G = model.generator(Z,Y)
D_real = model.discriminator(X, Y) 
D_gene = model.discriminator(G,Y)
  
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels = tf.ones_like(D_gene)))
        
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                  scope = 'discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                  scope = 'generator')
train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                             var_list = vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                             var_list = vars_G)


In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

total_epoch = 50
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = np.reshape(batch_xs,[-1,28,28,1])
        noise = model.get_noise(batch_size)

        _, cost1 = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, cost2 = sess.run([train_G, loss_G],
                                 feed_dict={Y: batch_ys, Z: noise})
        loss_val_D += cost1/total_batch
        loss_val_G += cost2/total_batch
   
    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

        
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = model.get_noise(sample_size)
        samples = sess.run(G,
                           feed_dict={Y: mnist.test.labels[:sample_size],
                                      Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/cnn_mnist{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('done')

Epoch: 0000 D loss: 0.6278 G loss: 2.934
Epoch: 0001 D loss: 1.268 G loss: 6.338
Epoch: 0002 D loss: 1.877 G loss: 8.862
Epoch: 0003 D loss: 2.414 G loss: 11.5
Epoch: 0004 D loss: 2.907 G loss: 14.24
Epoch: 0005 D loss: 3.372 G loss: 17.19
Epoch: 0006 D loss: 3.769 G loss: 20.35
Epoch: 0007 D loss: 4.103 G loss: 23.74
Epoch: 0008 D loss: 4.398 G loss: 27.39
Epoch: 0009 D loss: 4.696 G loss: 31.14
Epoch: 0010 D loss: 5.058 G loss: 35.02
Epoch: 0011 D loss: 5.769 G loss: 38.23
Epoch: 0012 D loss: 6.532 G loss: 40.65
Epoch: 0013 D loss: 7.381 G loss: 42.72
Epoch: 0014 D loss: 8.286 G loss: 44.59
Epoch: 0015 D loss: 9.254 G loss: 46.18
Epoch: 0016 D loss: 10.29 G loss: 47.61
Epoch: 0017 D loss: 11.37 G loss: 49.15
Epoch: 0018 D loss: 12.45 G loss: 50.5
Epoch: 0019 D loss: 13.56 G loss: 51.73
Epoch: 0020 D loss: 14.69 G loss: 52.96
Epoch: 0021 D loss: 15.84 G loss: 54.14
Epoch: 0022 D loss: 16.99 G loss: 55.43
Epoch: 0023 D loss: 18.13 G loss: 56.62
Epoch: 0024 D loss: 19.3 G loss: 57.71
Ep